Início

In [1]:
#ITR são consultas trimestrais
import pandas as pd
import requests
import os
import zipfile

In [2]:
diretorio_atual = os.getcwd()
if diretorio_atual == 'd:\OneDrive\Documentos\.TCC USP\PyIbov\dados_cvm\ITR':
    pass
else:
    os.chdir(f"{diretorio_atual}/dados_cvm/ITR")

In [3]:
anos = range(2018,2023)
url_base = 'https://dados.cvm.gov.br/dados/cia_aberta/doc/itr/DADOS/'

for ano in anos:
    download = requests.get(url_base+f'itr_cia_aberta_{ano}.zip')

    open(f'itr_cia_aberta_{ano}.zip', 'wb').write(download.content)

try:
    os.remove('filtro_dreITR.csv')
except:
    pass

print(pd.Timestamp.now())

2022-09-28 17:47:09.211156


Construção e formatação da base de dados

In [4]:
pd.Timestamp.now()
lista_demonstracoes = []
diretorio_atual = os.getcwd()
for arquivo in os.listdir(diretorio_atual):
    if arquivo == '.ipynb_checkpoints':
        pass
    else:
        arquivo_zip = zipfile.ZipFile(arquivo)

        for planilha in arquivo_zip.namelist():
            demonstracao = pd.read_csv(arquivo_zip.open(planilha),sep=';', encoding='ISO-8859-1',
                                        dtype={'ORDEM_EXERC':'category'})
            lista_demonstracoes.append(demonstracao)
pd.Timestamp.now()

Timestamp('2022-09-28 17:47:54.032191')

In [5]:
base_dados = pd.concat(lista_demonstracoes)
base_dados[:5]
#pd.Timestamp.now()

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,CATEG_DOC,ID_DOC,DT_RECEB,LINK_DOC,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC,COLUNA_DF
0,00.000.000/0001-91,2018-03-31,1,BCO BRASIL S.A.,1023,ITR,74121.0,2018-05-10,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00.000.000/0001-91,2018-06-30,1,BCO BRASIL S.A.,1023,ITR,76836.0,2018-08-09,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00.000.000/0001-91,2018-09-30,1,BCO BRASIL S.A.,1023,ITR,78776.0,2018-11-08,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00.000.208/0001-00,2018-03-31,1,BRB BCO DE BRASILIA S.A.,14206,ITR,74736.0,2018-05-15,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00.000.208/0001-00,2018-06-30,1,BRB BCO DE BRASILIA S.A.,14206,ITR,77660.0,2018-08-30,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
base_dados[['con_ind','tipo_dem']] = base_dados['GRUPO_DFP'].str.split('-',expand=True)
pd.Timestamp.now()

Timestamp('2022-09-28 17:49:10.413043')

In [7]:
base_dados['con_ind'] = base_dados['con_ind'].str.strip()
pd.Timestamp.now()

Timestamp('2022-09-28 17:49:17.810483')

In [8]:
base_dados['tipo_dem'] = base_dados['tipo_dem'].str.strip()
pd.Timestamp.now()

Timestamp('2022-09-28 17:49:31.526481')

In [9]:
base_dados = base_dados[base_dados['ORDEM_EXERC'] != 'PENÚLTIMO']
pd.Timestamp.now()

Timestamp('2022-09-28 17:51:20.523190')

In [10]:
lista_dem = base_dados['tipo_dem'].unique()
#base_dados['tipo_dem'].unique()
print(lista_dem)
pd.Timestamp.now()

[nan 'Balanço Patrimonial Ativo' 'Balanço Patrimonial Passivo'
 'Demonstração do Fluxo de Caixa (Método Direto)'
 'Demonstração do Fluxo de Caixa (Método Indireto)'
 'Demonstração das Mutações do Patrimônio Líquido'
 'Demonstração de Resultado Abrangente' 'Demonstração do Resultado'
 'Demonstração de Valor Adicionado']


Timestamp('2022-09-28 17:53:04.852487')

In [11]:
lista_empresas = base_dados['DENOM_CIA'].unique()
print(lista_empresas)
pd.Timestamp.now()

['BCO BRASIL S.A.' 'BRB BCO DE BRASILIA S.A.'
 'CENTRAIS ELET BRAS S.A. - ELETROBRAS'
 'COMPANHIA ENERGÉTICA DE BRASÍLIA - CEB' 'SHOPPING CENTER TACARUNA SA'
 'NEUMARKT TRADE AND FINANCIAL CENTER S/A' 'CIMS S.A.'
 'TELEC BRASILEIRAS S.A. TELEBRAS'
 'KOSMOS COMÉRCIO DE VESTUÁRIO S/A - EM RECUPERAÇÃO JUDICIAL'
 'ATOM EMPREENDIMENTOS E PARTICIPAÇÕES S.A.'
 'BNDES PARTICIPACOES S.A. - BNDESPAR' 'BANCO INTER S.A.' 'ELETRON S.A.'
 'SERRA AZUL WATER PARK SA' 'MAGNESITA MINERAÇÃO S.A.'
 'STATKRAFT ENERGIAS RENOVAVEIS S.A.' 'INVESTCO S.A.'
 'LITEL PARTICIPACOES S.A.' 'AMERICANAS S.A.'
 'CONC RODOVIA PRES. DUTRA S.A.' 'ENERGISA S.A.'
 'SANTA CATARINA PART INVEST SA' 'FERROVIA CENTRO-ATLANTICA S.A.'
 'CONC RIO-TERESOPOLIS S.A.' 'CAIXA ADM DIV PUB ESTADUAL SA'
 'CIELO S.A. -  INSTITUIÇÃO DE PAGAMENTO' 'NEOENERGIA S.A.'
 'ELETROBRÁS PARTICIPAÇÕES S.A. - ELETROPAR' 'BBM LOGÍSTICA S.A.'
 'ITAITINGA PARTICIPACOES S.A.' 'RUMO MALHA SUL S.A.'
 'GRUCAI PARTICIPACOES S.A.'
 'DASS NORDESTE CALÇADOS E ARTIG

Timestamp('2022-09-28 17:57:03.694302')

In [12]:
lista_conta = base_dados['DS_CONTA'].unique()
print(lista_conta)
pd.Timestamp.now()

[nan 'Ativo Total' 'Ativo Circulante' ...
 'Receitas financeiras (despesas financeiras)'
 'Provisão (reversão) de contingência'
 'Receitas relativas ativos próprios']


Timestamp('2022-09-28 17:59:37.546451')

In [13]:
filtro = ['IRB - BRASIL RESSEGUROS S.A.']
#'BCO BRASIL S.A.', 'BRB BCO DE BRASILIA S.A.',
filtro_dre = base_dados[base_dados.DENOM_CIA.isin(filtro)
                        #&(base_dados['tipo_dem']=='Demonstração de Resultados')
                        ]

In [17]:
lista_conta = filtro_dre['DS_CONTA'].unique()
lista_dem = filtro_dre['tipo_dem'].unique()
filtro_dre['base'] = 'itr'

C:\Users\olive\AppData\Local\Temp\ipykernel_12760\2073703591.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro_dre['base'] = 'itr'


In [18]:
campos = ['Imposto de Renda e Contribuição Social sobre o Lucro'
            ,'Receitas das Operações'
            ,'Lucro Líquido do Período'
            ,'Lucro Líquido Consolidado do Período'
            ,'Lucro/Prejuízo Consolidado do Período'
            ,'Lucro/Prejuízo do Período'
            ,'Depreciação, Amortização e Exaustão']
filtro_dre2 = filtro_dre[filtro_dre.DS_CONTA.isin(campos)]

In [19]:
filtro_dre.to_csv('filtro_dreITR.csv',sep=';')
pd.Timestamp.now()


Timestamp('2022-09-28 23:36:23.462522')